# Изучение воронки продаж в приложении для продажи продуктов питания

Учебная работа. Самостоятельный исследовательский проект в рамках курса "Аналитик данных" Яндекс.Практикума  
Спринт 10. Сборный проект №2.

Без output (не сохранился файл с данными)
 
## Общая информация о проекте
### Описание задания
Нужно изучить поведение пользователей мобильного приложения интернет-магазина продуктов питания. Проанализировать результаты А/А/В теста и сделать выводы о том, какой шрифт лучше (старый или новый). 

###  Цель
1. Изучить воронку продаж, определить, сколько пользователей доходит до покупки, на каких этапах "застревают" пользователи, которые не доходят до покупки.
2. Проанализировать результаты А/А/В теста с новым шрифтом

###  Задачи
1. Подготовить данные: проверить корректность данных в столбцах, проверить на наличие дубликатов, ошибок. 
2. Изучить данные. При необходимости изменить типы данных в столбцах, переименовать столбцы и создать недостающие для анализа.  
3. Изучить временной промежуток, за который есть логи. Отсеять старые данные.
4. Построить воронку продаж. Рассчитать долю пользователей, которая доходит от первого собития до оплаты, найти шаг воронки, на котором отсеивается наибольшее количество пользователей.
5. Проанализировать различия в поведении пользователей в А/А эксперименте. Селать вывод о том, корректно ли сделано разделение на группы.
6. Проанализировать различия в поведении пользователей в А/А/В и А/В экспериментах.


###  Данные
Данные получены в виде файла: `logs_exp.csv`.

###  Описание данных
  
**Структура `logs_exp.csv`**  
- `EventName` — название события;
- `DeviceIDHash` — уникальный идентификатор пользователя;
- `EventTimestamp` — время события;
- `ExpId` — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.

## Загрузка и знакомство с данными

Загружаем библиотеки:

In [ ]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem()
import datetime as dt
import matplotlib.pyplot as plt
from scipy import stats as st
import math
import seaborn as sns
import re
from plotly import graph_objects as go
from statsmodels.stats.proportion import proportions_ztest

import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")

Создаём библиотеку стилей:

In [ ]:
sns.set_style("ticks")

class Color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

Загружаем файл с данными:

In [ ]:
data = pd.read_csv('/datasets/logs_exp.csv', sep=None)

### Обзор и подготовка данных
Выведем первые 5 строк таблицы `data`:

In [ ]:
display(data.head())

Проверим данные о столбцах:

In [ ]:
data.info()

Присвоим столбцам более понятные названия:

In [ ]:
data.columns = ['event', 'user_id', 'date_time', 'test_group']
data.columns

Изменим тип данных в столбце `user_id` на `object` (это не обязательно, но так удобнее будет применить к нему функцию для изучения данных).

In [ ]:
data['user_id'] = data['user_id'].astype('object')

Изменим тип данных в столбце `date_time`:

In [ ]:
data['date_time'] = pd.to_datetime(data['date_time'], unit='s')
display(data.head(2))

Добавим в таблицу столбец с датами.

In [ ]:
data['date'] = data['date_time'].astype('datetime64[D]')
display(data.head(2))

Заменим численные значения в столбце `test_group` на категориальные "А1", "А2" и "В".

In [ ]:
data['test_group'] = data['test_group'].astype('str')
data['test_group'] = data['test_group'].replace('246', 'A1')
data['test_group'] = data['test_group'].replace('247', 'A2')
data['test_group'] = data['test_group'].replace('248', 'B')

print(data['test_group'].unique())
data['test_group'] = data['test_group'].astype('category')

In [ ]:
data.info()

Пропусков нет, тип данных в столбцах правильный. Проверим данные на наличие дубликатов:

In [ ]:
print('Дублирующихся строк:', data.duplicated().sum())

Удаляем дублирующиеся строки и делаем проверку

In [ ]:
data = data.drop_duplicates()
print('Дублирующихся строк:', data.duplicated().sum())

### Вывод
Проверены и исправлены типы данных, переименованы столбцы, приведены к более удобному формату названия тестовых групп, добавлен столбец с датами.

## Знакомство с данными


Для знакомства с данными используем функцию `column_details`.

In [ ]:
def column_details(data, column, bins, rows):
    """Функция для первичного знакомства с данными в столбце.
    Выводит название столбца, тип данных, описательную статистику, процент пропущенных значений, 
    количество уникальных значений, частотную гистограмму.
    Принимает аргументы:
        - название таблицы;
        - название столбца;
        - количество корзин для гистограммы
        - количество уникальных значений, которое нужно распечатать"""
    
    """Печатаем название столбца"""
    print(Color.BOLD + Color.UNDERLINE + 'Изучение данных в столбце ' + column + Color.END)
    print()
    
    """Печатаем тип данных в столбце"""
    dtype = data[column].dtypes
    print('{: <25}'.format('Тип данных:'), dtype)

    """Считаем  и выводим количество и процент пропущенных значений"""
    na_count = data[column].isna().sum()
    print('{: <25}'.format('Пропущенных значений:'), na_count)    
    
    lenth = len(data)
    na_part = na_count / lenth
    print('{: <25}'.format('Пропущенных значений:'), na_part)   
        
    """Считаем количество повторяющихся значений"""
    duplicated = data[column].duplicated().sum()
    print('{: <25}'.format('Повторяющихся значений:'), duplicated)   
    
    """Считаем количество уникальных значений"""
    unique = len(data[column].unique())
    print('{: <25}'.format('Уникальных значений:'), unique)   
    print()
    
    """В зависимости от типа данных выводим информацию о столбце"""
    
    if dtype == 'int64' or dtype == 'float64':
        """Если столбец содержит числовые данные, печатаем графики и описательную статистику""" 
        
        """Выводим описательную статистику по столбцу"""
        print(Color.BOLD + Color.BLUE + 'Описательная статистика' + Color.END)
        print()
        print(data[column].describe())
        print()           
        
        """Выводим частотную гистограмму"""
        min = data[column].min()
        max = data[column].max()
        title = 'Распределение значений столбца ' + column
        data[column].hist(bins=bins, range = (min, max))        
        plt.title(title)
        plt.show()

        """Выводим диаграмму размаха"""
        title = 'Диаграмма размаха значений столбца ' + column
        data.boxplot(column=column)
        plt.title(title)
        plt.show()
    
    elif dtype == bool or dtype == 'object':
        """Если столбец имеет тип bool, печатаем сколько строк имеет значение True и False,
        для типа object количество для каждого из уникальных значений"""
        
        print(Color.BOLD + Color.BLUE + 'Уникальные значения:' + Color.END)
        top_unique = data[column].value_counts().reset_index().head(rows)
        top_unique.columns = [column, 'count']
        print(top_unique)
    
    elif dtype == 'datetime64[ns]':
        """Если столбец имеет тип datetime, печатаем дату и время 
        первой и последней записи и строим частотную гистограмму"""
        print('Первая запись:', data[column].min())
        print('Последняя запись:', data[column].max())
        
        """Выводим частотную гистограмму"""
        min = data[column].min()
        max = data[column].max()
        title = 'Распределение значений столбца ' + column
        data[column].hist(bins=bins, range = (min, max))
        plt.xticks(rotation=90)
        plt.title(title)
        plt.show()


### Типы событий (столбец `event`)

Изучим данные в столбце `event`.

In [ ]:
column_details(data, 'event', 50, 5)

Событий всего 5: просмотр главного экрана, просмотр страницы предложения/товара, просмотр корзины, страница успешного платежа и просмотр справки. Названия событий длинные и непонятные. Заменим названия на более простые.

In [ ]:
print(data['event'].unique())

In [ ]:
# data['event'] = data['event'].astype('str')
data['event'] = data['event'].replace('MainScreenAppear', 'Главный экран')
data['event'] = data['event'].replace('OffersScreenAppear', 'Выбор товара')
data['event'] = data['event'].replace('CartScreenAppear', 'Корзина')
data['event'] = data['event'].replace('PaymentScreenSuccessful', 'Успешная оплата')
data['event'] = data['event'].replace('Tutorial', 'Справка')

print(data['event'].unique())

Так понятнее и на графиках будет выглядеть лучше.  
Построим столбчатую диаграмму по типам событий.

In [ ]:
event_types = data.groupby(['event'])[
    'user_id'].count().reset_index().sort_values(by='user_id', ascending=False)

sns.barplot(y='event', x='user_id', data=event_types).set(
    title='Количество событий по типам \n',
    xlabel='Количество событий',
    ylabel='Тип события')
plt.show()

Чаще всего пользователи посещают главный экран. Следующий шаг - экран выбора товара, дальше экран корзины и успешной покупки. На первый взгляд похоже, что больше всего пользователей теряется при переходе с главного экрана на экран выбора товара!  
А в справку заходит совсем мало пользователей.

**Вывод:**  
Названия событий переименованы.
Всего в данных встречается 5 типов событий. Чаще всего пользователи посещают главный экран. Следующий шаг - экран выбора товара, дальше экран корзины и успешной покупки. На первый взгляд похоже, что больше всего пользователей теряется при переходе с главного экрана на экран выбора товара! В справку заходит совсем мало пользователей.

### Данные о пользователях (столбец `user_id`)

Изучим данные в столбце `user_id`

In [ ]:
column_details(data, 'user_id', 50, 5)

Уникальных пользователей 7551, некоторые из них весьма активны: за исследуемый период совершили больше 1000 действий, зафиксированных как событие.

Рассчитаем описательную статистику и построим частотную гистограмму по количеству событий на одного пользователя:

In [ ]:
events_per_user = data.groupby('user_id')['event'].count()

sns.boxplot(events_per_user, fliersize = 0).set(
    title='Количество событий на одного пользователя \n',
    xlabel='Количество событий')
plt.xlim(-10, 100)
plt.show()

events_per_user.describe()

Минимальное количество событий на одного пользователя - одно (видимо, заглянул на главную страницу, не заинтересовался, и больше не появлялся).  
Разница между медианным количеством событий (20) и средним (32), а также стандартное отклонение - очень большие: распределение сильно скошено влево.  
Для большинства пользователей (75%) совершают не более 37 действий, зафиксированных как событие.  
Единичные пользователи совершают аномально большое количество действий (больше 1000). Не очень понятно, кто эти пользователи. Могут они быть реальными пользователями или это тестеры и поисковые боты, которые попали в логи? Это выбросы, которые могли бы сильно искажать картину, но в данном случае они нам не мешают. 

Посчитаем 95-й и 99-й перцентиль по количеству действий на пользователя: 

In [ ]:
print(np.percentile(events_per_user, [95, 99])) 

5% пользователей совершали больше 89 действий и 1 процент пользователей совершал больше 200 действий. Предположим, что наши пользователи - очень дотошные и внимательные люди, совершают много покупок и долго сомневаются перед каждой. В таком случае можно отсеять пользователей, совершивших более 200 действий.

Составим список пользователей, которые совершили более двухсот действий:

In [ ]:
events_per_user = events_per_user.reset_index()
abnormalUsers = events_per_user[events_per_user['event'] > 200]['user_id'].unique()
print('Аномальных пользователей:', len(abnormalUsers))

Таких пользователей оказалось 76. Исключим данные о действиях аномальных пользователей, построим диаграмму размаха и рассчитаем описательную статистику по отфильрованным данным:

In [ ]:
data_filtered = data[np.logical_not(data['user_id'].isin(abnormalUsers))]
print('Доля отсеянных данных: {:.2%}'.format(1 - len(data_filtered)/len(data)))

1% пользователей генерировал 14% данных! Это не удивительно, учитывая, что большая часть пользователей совершала от 10 до 30 действий, а аномальные - до двух тысяч.  
Сохраним отфильтрованные данные в таблицу `data`.

In [ ]:
data = data_filtered

**Вывод:**  
Изучены данные о пользователях, найдены пользователи, совершающие аномально большое количество действий. 1% пользователей исключён из данных (что составило 14% данных).

### Данные времени события (столбец `date_time`)

Изучим данные в столбце `date_time`

In [ ]:
column_details(data, 'date_time', 14, 5)

Запись о первом по времени событии сделана 25 июля 2019, но после этого до конца июля почти никаких записей нет.  
1 августа количество событий вырастает до 20 тысяч, а следующие 6 дней держится на уровне от 30 до 40 тысяч. Похоже, что активный период сбора данных длился 7 дней.  
Проверим это предположение: найдём дату первого посещения главной страницы для каждого пользователя.

In [ ]:
first_visit_date = data[data['event'] == 'Главный экран'].pivot_table(
    index='user_id',
    values='date',
    aggfunc='first'
).reset_index()

fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(first_visit_date['date'], bins=14)
ax.set_title('Количество пользователей, впервые посетивших главную страницу (по дням)')
ax.set_xlabel('дата')
ax.set_ylabel('количество пользователей')
plt.xticks(rotation=90)
plt.show()

Исключим засипи о событиях с 25 по 31 июля:

In [ ]:
data_august = data.query('date >= "2019-08-01"')
print('Доля отсеянных данных: {:.2%}'.format(1 - len(data_august)/len(data)))

Для отсеянных данных - всего 1,3% (за неделю).  
Сохраним отфильтрованные данные в таблицу `data`.

Считаем потери по количеству пользвателей:

In [ ]:
print('Доля отсеянных пользователей: {:.2%}'.format(1 - data_august['user_id'].nunique()/data['user_id'].nunique()))

По числу пользователей потери совсем несущественные, 0.23%.
Сохраним данные в таблицу `data`.

In [ ]:
data = data_august

**Вывод:**  
Изучены данные о дате: в данных есть записи о действиях пользователей за 2 недели (с 25 июля по 7 августа 2019 года), но за первую неделю записей очень мало, похоже, что они были выгружены случайно. Для анализа сохранены данные из основного массива (неделя с 1 по 7 августа 2019), доля отсейнных данных составила 1,3%.

### Данные об группах А/А/В теста (столбец `test_group`)

Посчитаем, сколько записей и сколько пользователей попало в каждую группу:

In [ ]:
data.groupby('test_group').agg({'user_id' : 'nunique', 'event' : 'count'})

Количество записей и пользователей в группах примерно одного порядка.  
Проверим корректность разделения пользователей на группы:

In [ ]:
usersA1 = data.query('test_group == "A1"')['user_id']
usersA2 = data.query('test_group == "A2"')['user_id']
usersB = data.query('test_group == "B"')['user_id']

display(any(usersA1.isin(usersA2)))
display(any(usersA1.isin(usersB)))
display(any(usersA2.isin(usersB)))

Данных о пользователях, попавших сразу в несколько групп, нет.  

**Вывод:**  
Пользователи разделены на группы корректно (группы близки по размерам и активности, пользователей, попавших в несколько групп сразу, нет).

### Вывод

1. В столбце `event` названия событий заменены на полнуе русские.  
Всего в данных встречается 5 типов событий. Чаще всего пользователи посещают главный экран. Следующий шаг - экран выбора товара, дальше экран корзины и успешной покупки. На первый взгляд похоже, что больше всего пользователей теряется при переходе с главного экрана на экран выбора товара! В справку заходит совсем мало пользователей.  

2. Изучены данные о пользователях (столбец `user_id`), найдены пользователи, совершающие аномально большое количество действий. 1% пользователей исключён из данных (что составило 14% данных).

3. Изучены данные о дате (столбец `date_time`): в данных есть записи о действиях пользователей за 2 недели (с 25 июля по 7 августа 2019 года), но за первую неделю записей очень мало, похоже, что они были выгружены случайно. Для анализа сохранены данные из основного массива (неделя с 1 по 7 августа 2019), доля отсейнных данных составила 1,3%, доля отсеянных пользователей составила 0.23%.

4. Изучены данные о тестовых группах в столбце `test_group`. Пользователи разделены на группы корректно (группы близки по размерам и активности, пользователей, попавших в несколько групп сразу, нет).

## Анализ данных
### Воронка событий

Построим воронку событий и рассчитаем конверсию для каждого из шагов, чтобы понять, на каком шаге теряется больше всего пользователей.

#### Типы событий
Построим ещё раз график количества собитий по типам:

In [ ]:
sns.barplot(y='event', x='user_id', data=event_types).set(
    title='Количество событий по типам \n',
    xlabel='Количество событий',
    ylabel='Тип события')
plt.show()

Посчитаем, сколько уникальных пользователей совершали каждое событие:

In [ ]:
users_per_event = data.groupby('event')['user_id'].nunique().reset_index().sort_values('user_id', ascending=False)


sns.barplot(y='event', x='user_id', data=users_per_event).set(
    title='Количество событий по типам \n',
    xlabel='Количество событий',
    ylabel='Тип события')
plt.show()

Больше всего по прежнему посещений главной страницы, но доля других событий значительно выросла.  
Посчитаем долю пользователей, которые хотя бы раз совершали событие:

In [ ]:
unique_users = data['user_id'].nunique()
users_per_event['part'] = users_per_event['user_id']/unique_users
display(users_per_event.round(2))


98% пользователей посещали главную страницу. Либо у нас остались "хвосты" пользователей, которые посетили главную страницу раньше 1 августа, либо были пользователи, пришедшие на страницу товара по внешней ссылке (но тогда их, скорее всего, было бы больше двух процентов). Скорее всего, это "хвосты".   
На страницу предложений переходит всего 61%. Пока этот шаг выглядит самым слабым звеном (на страницу предложений заходит на треть меньше уникальных пользователей, чем на главную).  
С переходом по следующим шагам всё пока выглядит довольно хорошо (разница в доле пользователей 11% и 3%).  
В справку смотрит 11% пользователей. Кажется, это довольно много. Интересно, что они там ищут?

#### Порядок событий

Пользователю не обязательно заходить в справку, чтобы совершить покупку. 
Логический порядок остальных событий соответствует частоте встречаемости: 
 - зайти на главную страницу,
 - перейти на страницу выбора предложений,
 - перейти в корзину,
 - оплатить покупку.

Строим воронку событий:

In [ ]:
users_per_event = data.pivot_table(
    index='event',
    values='user_id',
    aggfunc=lambda x: x.nunique()
).sort_values(by='user_id', ascending=False).reset_index()
users_per_event.columns=['event', 'active_users']
users_per_event = users_per_event.drop(users_per_event[users_per_event['event'] == 'Справка'].index)

fig = go.Figure(
    go.Funnel(
        y=users_per_event['event'], x=users_per_event['active_users']))
fig.update_layout(height=500, title=dict(
    text='<b>Воронка событий в приложении для покупки продуктов питания</b>', x=0.5, y=0.90, font=dict(
        family="Arial",
        size=20,
        color='#000000')))
fig.show() 

#### Конверсия по шагам

Рассчитаем для каждого шага долю пользователей, перешедших на следующий шаг:

In [ ]:
"""Добавляем строку all_users и сдвигаем строки на 1 относительно индекса"""
all_users = pd.DataFrame(columns=['event', 'active_users'], data=[['Все пользователи', unique_users]])
users_funnel = users_per_event.append(all_users).sort_values(by='active_users', ascending=False)[:-1].reset_index(drop=True)

"""Объединяем исходные данные и сдвинутые"""
users_funnel = users_per_event.join(users_funnel, rsuffix='_shirted')
users_funnel = users_funnel.drop('event_shirted', axis=1)

"""Рассчитываем конверсию"""
users_funnel['funnel'] = users_funnel['active_users'] / users_funnel['active_users_shirted']
funnel = users_funnel[['event', 'funnel']].set_index('event', drop=True).reset_index()
funnel = funnel.style.format({
    'funnel': '{:,.0%}'.format
})
display(funnel)

Как мы и видели раньше, больше всего пользователей теряется при переходе с главного экрана на страницу выбора товара, 38% пользователей уходят, так и не увидев предложение!  
На следующих этапах конверсия гораздо лучше: 81% пользователей, просмотревших товар, переходят в корзину (то есть большинству товар нравится) и 95% пользователей, просмотревших корзину, завершают покупку успешной оплатой.

Найдём долю пользователей, успешно завершивших покупку, от общего числа пользователей:

In [ ]:
print(
    'Доля пользавателей, дошедших от первого события до оплаты: {:.0%}'.format(
        users_per_event.loc[3, 'active_users']/unique_users))

От первого просмотра до оплаты доходит 46% пользователей. Это выглядит как неплохой результат, но, учитывая, сколько пользователей уходит, даже не посмотрев на предложение, этот процент мог бы быть ещё лучше.  
Интересно, какую долю от всех потерянных пользователей составляют пользователи, потерянные при переходе с главной страницы?

In [ ]:
all_lost_users = (unique_users-users_per_event.loc[3, 'active_users'])
MainScreen_lost_users = (unique_users-users_per_event.loc[1, 'active_users'])
part_of_MainScreen_lost_users = MainScreen_lost_users / all_lost_users
print('Всего потерянных пользователей: {:.0%}'.format(all_lost_users/unique_users))
print('Пользователей, потерянных на переходе с главной страницы (от общего количества пользователей): {:.0%}'
      .format(MainScreen_lost_users/unique_users))
print('Всего потерянных пользователей: {:.0%}'.format(part_of_MainScreen_lost_users))

74% всех "застрявших" пользователей потеряны на переходе с "Главного экрана" на страницу "Выбор товара". С главным экраном опредёлённо что-то не так.

#### Вывод

Конверсия на этапе "просмотр главного экрана" / "просмотр товара" - 62%.    
Конверсия на этапе "просмотр товара" / "просмотр корзины" 81%,   
а на этапе "просмотр корзины" / "оплата" 95%.  
Товар нравится пользователям (его выбирают и оплачивают), но до оплаты доходит только 46%, при этом больше всего пользователей (39% от всех пользователей и 74% от всех потерянных пользователей) теряется при переходе к странице выбора товара.

### Анализ результатов эксперимента

У нас есть результаты А/А/В теста для новых шрифтов в приложении. 

Сначала изучим результаты А/А теста, чтобы проверить корректность проведения тестирования, затем сравним экспериментальную группу В с граппами А1, А2 и совмещёнными данными обеих контрольных групп. Проверим, есть ли различия между группами в конверсии по шагам: переход на главную страницу, переход на страницу выбора товара, переход в корзину, переход на страницу успешной оплаты.  

Для рассчёта статистической значимости различий применим z-тест, проверку выполним при помощи функции `proportions_ztest` библиотеки `statsmodels`, а для сравнения групп используем самописную функцию.

#### Подготовка данных и функции для анализа результатов А/А/В теста

Найдём количество уникальных пользователей в каждой группе и сохраним в переменные:

In [ ]:
group_size_a1 = data[data['test_group'] == 'A1']['user_id'].nunique()
group_size_a2 = data[data['test_group'] == 'A2']['user_id'].nunique()
group_size_b = data[data['test_group'] == 'B']['user_id'].nunique()

print('Контрольная группа А1: {} уникальных пользователей'.format(group_size_a1))
print('Контрольная группа А2: {} уникальных пользователей'.format(group_size_a2))
print('Экспериментальная/ группа В: {} уникальных пользователей'.format(group_size_b))

Раcсчитаем воронку событий по группам теста:

In [ ]:
test_data = data.pivot_table(
    index='event',
    columns='test_group',
    values='user_id',
    aggfunc=lambda x: x.nunique()
)
test_data.columns = ['A1', 'A2', 'B']
test_data = test_data.sort_values(by='B', ascending=False).drop('Справка').reset_index()
display(test_data)

Добавим в строки данные о количестве пользователей в каждой группе, чтобы рассчитать конверсию шагов:

In [ ]:
test_data['A1_group_size'] = group_size_a1
test_data['A2_group_size'] = group_size_a2
test_data['B_group_size'] = group_size_b

users_funnel = test_data

display(users_funnel)

Добавим столбцы для совмещённой контрольной группы 

In [ ]:
"""Добавляем столбец А1А2 и А1А2_shifted с объединёнными данными обеих контрольных групп"""
users_funnel['A1A2'] = users_funnel['A1'] + users_funnel['A2']
users_funnel['А1А2_group_size'] = users_funnel['A1_group_size'] + users_funnel['A2_group_size']

display(users_funnel)

Мы будем делать сравнение групп А1/А2, А1/В, А2/В и (А1+А2)/В, поэтому будет удобнее сделать функцию.  
Функция `ztest` принимает таблицу с данными воронки событий и выводит значимость различий между группами по каждому шагу воронки.

In [ ]:
def ztest(data, alpha, correction, title):
    """Функция для расчёта статистической значимости различий конверсии для воронки событий двух выборок. 
    Выводит значимость различий между группами по шагам воронки событий.
    
    Принимает аргументы:
    - data - название таблицы с данными;
    - alpha - уровень значимости для статистических критериев;
    - correction - количество проверяемых гипотез для поправки на множественные сравнения;
    - title - заголовок."""
    
    
    """Печатаем заголовок"""
    print(Color.BOLD + Color.BLUE + title + Color.END)
    print()
    
    """Печатаем нулевую и альтернативную гипотезы"""
    print(Color.BOLD + 'Считаем статистическую значимость различий. Z-тест' + Color.END)
    print(Color.GREEN + 
          'H₀: доля от общего количества пользователей, совершивших переход к следующему шагу \n' +
          'в первой выборке и соответствующая доля пользователей во второй выборке статистически не различается\n' 
          + Color.RED + 
          'H₁: доля от общего количества пользователей, совершивших переход к следующему шагу \n' +
          'в первой выборке статистически отличается от соответствующей доли пользователей во второй выборке' + Color.END)
    
    """В цикле рассчитываем z-test по строкам таблицы"""
    
    for index, row in data.iterrows():
        print('__________________________________________________________________________')
        print()
        print('Шаг {}:'.format(index+1),
            'Переход на страницу ' + Color.BOLD + 
            '"{}"'.format(row[0]) + Color.END)
        stat, pval = proportions_ztest([row[1], row[2]], [row[3], row[4]])
        
        """Сравниваем p-value и уровень значимости и выводим результат"""
        if pval < alpha/correction:
            print(Color.RED + 'p-value = {0:.3f}'.format(pval) + 
                  '\nОтвергаем нулевую гипотезу: разница статистически значима' + Color.END)
        else:
            print(Color.GREEN + 'p-value = {0:.3f}'.format(pval) + 
                  '\nНе получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя' + Color.END)

    print()

#### Анализ результатов А/А теста

Проанализируем результата теста А/А, чтобы проверить корректность проведения эксперимента при помощи функции `ztest`.   

В рамках проверки этой гипотезы мы проверяем 4 сравнения, поэтому сделаем поправку на множественные сравнения и разделим alpha на 16 (4 пары групп и 4 события).

In [ ]:
ztest(users_funnel[[
    'event', 'A1', 'A2', 'A1_group_size', 'A2_group_size']], 0.05, 16, 'Различия между контрольными группами А1 и А2')

Различий между двумя контрольными группами нет: разделение на группы сделано корректно. 

#### Анализ результатов А1/В, А2/В и А1А2/В тестов

Проверим, есть ли различия между контрольными группами и экспериментальной. В функцию передаются названия столбцов с размерами групп. Делитель для уровня значимости - 16.

In [ ]:
ztest(users_funnel[['event', 'A1', 'B', 'A1_group_size', 'B_group_size']
                  ], 0.05, 16, 'Различия между группой А1 и =экспериментальной группой В')
ztest(users_funnel[['event', 'A2', 'B', 'A2_group_size', 'B_group_size']
                  ], 0.05, 16, 'Различия между группой А1 и =экспериментальной группой В')
ztest(users_funnel[['event', 'A1A2', 'B', 'А1А2_group_size', 'B_group_size']
                  ], 0.05, 16, 'Различия между группой А1 и =экспериментальной группой В')

Ни одну нулевую гипотезу из 16 отвергнуть не удалось, вывод о различиях сделать нельзя. Нет оснований говорить о том, что новый шрифт как-то по-другому воспринимается пользователями и влияет на их поведение.

#### Вывод

Проанализированы результаты А/А/В теста.  
Различий между контрольными граппами не обнаружено: тест проведён корректно.  
Различий между экспериментальной и контрольными группами не обнаружено, нет оснований говорить о том, что новый шрифт влияет на поведение пользователей.

## Обзор проведённой работы и выводы

### Краткий обзор проведённой работы

1. Данные подготовлены к работе. Найдены и удалены дубликаты, столбцы приведены к правильным типам данных, исключены данные об аномальных пользователях, отсеяны неполные данные.  
2. Построена воронка событий и рассчитана конверсия для каждого шага воронки. Рассчитана доля посетителей, которые проходят весь путь от первого шага до покупки. Найден шаг, накотором теряется больше всего пользователей.  
3. Проанализированы результаты А/А/В теста.
4. Сделаны выводы.

### Выводы

#### Воронка событий и анализ конверсии

Всего в данных встречается 5 типов событий.  
Чаще всего пользователи посещают главный экран. Следующий шаг - экран выбора товара, экран корзины и успешной покупки. В справку заходит совсем мало пользователей. 

**Конверсия по шагам**  
Конверсия на этапе "просмотр главного экрана" / "просмотр товара" - 62%.    
Конверсия на этапе "просмотр товара" / "просмотр корзины" 81%,   
а на этапе "просмотр корзины" / "оплата" 95%.  

**Общая конверсия**  
Общая конверсия всех посетителей приложения в покупателей составляет 46%.

**Больше всего пользователей теряется на шаге**  
Больше всего пользователей (38%) теряется при переходе с главной страницы к странице выбора товара.

**Выводы и рекомендации**  
Товар нравится пользователям (его выбирают и оплачивают), но до оплаты доходит меньше половины пользователей. При этом большая часть потерянных пользователей даже не увидела предложения (39% от всех пользователей и 74% от всех потерянных пользователей).  
Стоит выяснить, почему пользователи уходят с главной страницы, с большой вероятность это должно привести к росту конверсии.

#### Результаты А/А/В теста

**Результаты А/А теста**  
Различий между контрольными граппами не обнаружено: тест проведён корректно.

**Результаты А/В теста**  
Различий между экспериментальной и контрольными группами не обнаружено, нет оснований говорить о том, что новый шрифт влияет на поведение пользователей.